<a href="https://colab.research.google.com/github/steven1174/Data_Science_Handbook/blob/main/04.Handling%20Missing%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Missing Data
The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of data missing. To make matters even more complicated,
different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python. Here and throughout the book, we’ll refer to missing
data in general as null, NaN, or NA values.
## Trade-Offs in Missing Data Conventions
A number of schemes have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies: using a mask that globally indicates missing values, or choosing a sentinel value that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with –9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented,
and may require extra (often non-optimized) logic in CPU and GPU arithmetic.

Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions. For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell to indicate a NA state.
## Missing Data in Pandas
The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for nonfloating-point data types.

Pandas could have followed R’s lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy. While R contains four basic data types, NumPy supports far more than this: for example, while R has a single integer type, NumPy supports fourteen basic integer types once you account for available precisions, signedness, and endianness of the encoding. Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types,likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays—that is, arrays that have a separate Boolean mask array attached for marking data as “good” or “bad.” Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance
makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point NaN value, and the Python None object. This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.
### None: Pythonic missing data

In [1]:
import pandas as pd
import numpy as np

In [2]:
vals1 = np.array([1,None,3,4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
  print("dtype =", dtype)
  %timeit np.arange(1E6, dtype=dtype).sum()
  print()


dtype = object
10 loops, best of 3: 68.1 ms per loop

dtype = int
100 loops, best of 3: 2.11 ms per loop



In [4]:
vals1.sum()

TypeError: ignored

### NaN: Missing numerical data

In [ ]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

In [ ]:
1 + np.nan

In [ ]:
0 * np.nan

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

### NaN and None in Pandas

In [ ]:
pd.Series([1, np.nan, 2, None])

In [ ]:
x = pd.Series(range(2), dtype=int)
x

In [ ]:
x[0] = None
x

<html>
<head>
<style>
table {
  font-family: Times New Roman, sans-serif;
  width: 100%;
}
td, th {
  border: 2px solid #dddddd;
  text-align: center;
  padding: 8px;
}
tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>

<body>
<table>
  <tr>  
    <th>Typeclass</th>
    <th>Conversion when storing NAs</th>
    <th>NA sentinel value</th>
  </tr>
  <tr>
    <td>floating</td>
    <td>No change</td>
    <td>np.nan</td>
  </tr>
  <tr>
    <td>object</td>
    <td>No change</td>
    <td>None or np.nan</td>
  </tr>
  <tr>
    <td>integer</td>
    <td>Cast to float64</td>
    <td>np.nan</td>
  </tr>
  <tr>
    <td>boolean</td>
    <td>Cast to object</td>
    <td>None or np.nan</td>
  </tr>
</table>

</body>
</html>

## Operating on Null Values
- isnull() : Generate a Boolean mask indicating missing values
- notnull() : Opposite of isnull()
- dropna() : Return a filtered version of the data
- fillna() : Return a copy of the data with missing values filled or imputed
### Detecting null values

In [5]:
data = pd.Series([1,np.nan,'hello',None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [6]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [7]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values

In [8]:
data.dropna()

0        1
2    hello
dtype: object

In [9]:
df = pd.DataFrame([[1, np.nan, 2],[2, 3, 5],[np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [10]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [11]:
df.dropna(axis = 1)

,2
0,2
1,5
2,6


In [14]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [16]:
df.dropna(axis = 1, how = 'all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [17]:
df.dropna(axis = 'rows',thresh= 3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Filling null values

In [18]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [19]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [20]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [21]:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [22]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [23]:
df.fillna(method='ffill',axis = 1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
